# Создание различных ML моделей для определения спам сообщений

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pickle

In [2]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from nltk.corpus import stopwords

In [3]:
import sys
sys.path.append("..")  # Добавление корневой папки в список путей поиска модулей.
from utils.preprocessing import count_tags, count_links, count_emojis, count_newlines, count_whitespaces, count_newlines, replace_tags, replace_links, remove_emojis, preprocess_text

2025-07-20 02:44:15,658 - INFO - Логгер успешно настроен. Логи записываются в файл: logs/2025-07-20_02-44-15.log


## Чтение обработанного датасета

In [4]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [5]:
df.head()

text  label  emojis  newlines  \
0  Добрый день! Отличается ли перечень необходимы...      0       0         0   
1              Узбекистан. Рассматриваются обе формы      0       0         0   
2  Здравствуйте, а как проходит поступление после...      0       0         0   
3                          Спасибо большое за ответ!      0       0         0   
4  Здравствуйте, а когда будет день открытых двер...      0       0         0   

   whitespaces  links  tags                                  text_preprocessed  
0            0      0     0  добрый день отличается ли перечень необходимых...  
1            0      0     0               узбекистан рассматриваются обе формы  
2            0      0     0  здравствуйте а как проходит поступление после ...  
3            0      0     0                           спасибо большое за ответ  
4            0      0     0  здравствуйте а когда будет день открытых двере...

## Подготовка выборок

In [6]:
texts = list(df['text_preprocessed'])  # Тексты
labels = list(df['label'])  # Метки (целевые значения)

In [7]:
additional_features = list(df[['emojis', 'newlines', 'whitespaces', 'links', 'tags']].values)  # Дополнительные числовые признаки

### Разделение на обучающую и тестовую выборки

In [8]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [9]:
X_train_num, X_test_num = train_test_split(
    additional_features, test_size=0.2, random_state=42, stratify=labels
)

### Подготовка стоп-слов

Загрузка английских и русских стоп-слов (`nltk`)

In [10]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
english_stopwords = set(ENGLISH_STOP_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\overklassniy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Объединение русских и английских стоп-слов

In [11]:
combined_stopwords = list(russian_stopwords.union(english_stopwords))

In [12]:
combined_stopwords

['seem',
 'seems',
 'thereupon',
 'уж',
 'may',
 'any',
 'от',
 'опять',
 'third',
 'да',
 'вас',
 'есть',
 'над',
 'can',
 'full',
 'во',
 'при',
 'sixty',
 'toward',
 'ourselves',
 'of',
 'thereafter',
 'ever',
 'mill',
 'ours',
 'already',
 'до',
 'become',
 'the',
 'those',
 'into',
 'put',
 'much',
 'за',
 'об',
 'co',
 'each',
 'много',
 'again',
 'been',
 'всех',
 'whereas',
 'перед',
 'as',
 'was',
 'onto',
 'please',
 'however',
 'throughout',
 'yet',
 'а',
 'could',
 'у',
 'from',
 'nine',
 'their',
 'only',
 'about',
 'зачем',
 'больше',
 'ему',
 'более',
 'один',
 'take',
 'вдруг',
 'том',
 'front',
 'am',
 'чтоб',
 'is',
 'fifteen',
 'hereafter',
 'вам',
 'formerly',
 'хоть',
 'так',
 'cant',
 'beside',
 'or',
 'until',
 'it',
 'behind',
 'beforehand',
 'со',
 'только',
 'inc',
 'though',
 'at',
 'конечно',
 'anywhere',
 'cry',
 'except',
 'еще',
 'себе',
 'next',
 'же',
 'три',
 'per',
 'им',
 'thin',
 'и',
 'during',
 'well',
 'myself',
 'has',
 'never',
 'hereupon',
 'н

Проверка на наличие слов `tag` и `link` в списке стоп-слов.

In [13]:
'TAG' in combined_stopwords, 'tag' in combined_stopwords, 'LINK' in combined_stopwords, 'link' in combined_stopwords

(False, False, False, False)

In [14]:
len(combined_stopwords)

469

### Подготовка текстовых и числовых признаков

#### Преобразование текста в числовой формат (TF-IDF)

In [15]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=combined_stopwords,
    min_df=2,
    sublinear_tf=True
)

In [16]:
text_features_train = vectorizer.fit_transform(X_train_text)
text_features_test = vectorizer.transform(X_test_text)

#### Масштабирование числовых признаков

In [17]:
numerical_features_train = pd.DataFrame(X_train_num, columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags'])
numerical_features_test = pd.DataFrame(X_test_num, columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags'])

scaler = StandardScaler()
numerical_features_train_scaled = scaler.fit_transform(numerical_features_train)
numerical_features_test_scaled = scaler.transform(numerical_features_test)

#### Объединение текстовых и числовых признаков

In [18]:
X_train = hstack([text_features_train, numerical_features_train_scaled])
X_test = hstack([text_features_test, numerical_features_test_scaled])

#### Дополнительная синтетическая генерация

In [19]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Обучение моделей

### Logistic Regression

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE 

In [20]:
model_lr = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Logistic Regression + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [21]:
model_lr_sm = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr_sm.fit(X_train_resampled, y_train_resampled)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Random Forest

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

In [22]:
model_rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Random Forest + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [23]:
model_rf_sm = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf_sm.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Naive Bayes

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [24]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

Обучение Naive Bayes

In [25]:
model_nb = MultinomialNB()
model_nb.fit(X_train_nc, y_train)

MultinomialNB()

### Naive Bayes + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка c SMOTE

Объединение текстов и числовых признаков без масштабирования

In [26]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

In [27]:
smote = SMOTE(random_state=42)
X_train_resampled_nc, y_train_resampled_nc = smote.fit_resample(X_train_nc, y_train)

In [28]:
model_nb_sm = MultinomialNB()
model_nb_sm.fit(X_train_resampled_nc, y_train_resampled_nc)

MultinomialNB()

## Оценка качества моделей

Примеры тестовых сообщений

In [29]:
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!",
    "3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1",
    "Выиграл 345к дало x3450\n\nИграл тут: @jet_casino_ibot"
]

Функция для предсказания на новых данных

> Эта же функция находится в `../utils/preprocessing.py`.

In [30]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text: (str) - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_newlines(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

### Logistic Regression

In [31]:
y_pred = model_lr.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95      6689
           1       0.98      0.93      0.95      7019

    accuracy                           0.95     13708
   macro avg       0.96      0.95      0.95     13708
weighted avg       0.96      0.95      0.95     13708

Confusion Matrix:
 [[6583  106]
 [ 525 6494]]


In [32]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-07-20 02:45:36,989 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-07-20 02:45:36,991 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-07-20 02:45:36,994 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-07-20 02:45:36,995 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-07-20 02:45:36,997 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-07-20 02:45:36,998 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-07-20 02:45:37,000 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-07-20 02:45:37,001 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-07-20 02:45:37,002 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9397379, 0.0602621]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0256614, 0.9743386]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.6112914, 0.3887086]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.0161926, 0.9838074]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9480703, 0.0519297]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.1

### Logistic Regression + SMOTE

In [33]:
y_pred = model_lr_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95      6689
           1       0.98      0.93      0.95      7019

    accuracy                           0.95     13708
   macro avg       0.96      0.95      0.95     13708
weighted avg       0.96      0.95      0.95     13708

Confusion Matrix:
 [[6586  103]
 [ 524 6495]]


In [34]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-07-20 02:45:37,067 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-07-20 02:45:37,067 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-07-20 02:45:37,070 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-07-20 02:45:37,070 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-07-20 02:45:37,072 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-07-20 02:45:37,073 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-07-20 02:45:37,075 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-07-20 02:45:37,076 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-07-20 02:45:37,078 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9438345, 0.0561655]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.024623, 0.975377]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.6084919, 0.3915081]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.016471, 0.983529]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9487687, 0.0512313]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.15697

### Random Forest

In [35]:
y_pred = model_rf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96      6689
           1       0.98      0.94      0.96      7019

    accuracy                           0.96     13708
   macro avg       0.96      0.96      0.96     13708
weighted avg       0.96      0.96      0.96     13708

Confusion Matrix:
 [[6558  131]
 [ 445 6574]]


In [36]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-07-20 02:45:38,211 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-07-20 02:45:38,212 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-07-20 02:45:38,226 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-07-20 02:45:38,227 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-07-20 02:45:38,236 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-07-20 02:45:38,236 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-07-20 02:45:38,246 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-07-20 02:45:38,246 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-07-20 02:45:38,253 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.95, 0.05]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.1409975, 0.8590025]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9490274, 0.0509726]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.06, 0.94]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8221515, 0.1778485]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.3116536, 0.6883464]



### Random Forest + SMOTE

In [37]:
y_pred = model_rf_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96      6689
           1       0.98      0.94      0.96      7019

    accuracy                           0.96     13708
   macro avg       0.96      0.96      0.96     13708
weighted avg       0.96      0.96      0.96     13708

Confusion Matrix:
 [[6556  133]
 [ 440 6579]]





In [38]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-07-20 02:45:39,537 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-07-20 02:45:39,538 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-07-20 02:45:39,549 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-07-20 02:45:39,550 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-07-20 02:45:39,558 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-07-20 02:45:39,558 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-07-20 02:45:39,566 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-07-20 02:45:39,567 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-07-20 02:45:39,574 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.94, 0.06]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.110801, 0.889199]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.98, 0.02]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.07, 0.93]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8523511, 0.1476489]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.3918687, 0.6081313]

Сообщение: D

### Naive Bayes

In [39]:
y_pred = model_nb.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.75      6689
           1       1.00      0.35      0.52      7019

    accuracy                           0.67     13708
   macro avg       0.80      0.68      0.63     13708
weighted avg       0.80      0.67      0.63     13708

Confusion Matrix:
 [[6686    3]
 [4555 2464]]





In [40]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-07-20 02:45:39,675 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-07-20 02:45:39,677 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-07-20 02:45:39,682 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-07-20 02:45:39,683 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-07-20 02:45:39,686 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-07-20 02:45:39,686 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-07-20 02:45:39,689 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-07-20 02:45:39,689 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-07-20 02:45:39,691 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9990613, 0.0009387]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9876264, 0.0123736]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9967593, 0.0032407]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.853816, 0.146184]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9999788, 2.12e-05]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.9930

2025-07-20 02:45:39,707 - INFO - Предобработка завершена. Итоговый текст: как же надоели эти сообщения про казино
2025-07-20 02:45:39,709 - INFO - Начало предобработки текста: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru
2025-07-20 02:45:39,709 - INFO - Предобработка завершена. Итоговый текст: добрый день для подачи документов необходимо пройти регистрацию здесь [LINK]
2025-07-20 02:45:39,711 - INFO - Начало предобработки текста: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
2025-07-20 02:45:39,712 - INFO - Предобработка завершена. Итоговый текст: добрый день для подачи документов необходимо пройти регистрацию здесь [LINK]
2025-07-20 02:45:39,714 - INFO - Начало предобработки текста: Поступление – это почти что казино! Лотерея!
2025-07-20 02:45:39,715 - INFO - Предобработка завершена. Итоговый текст: поступление это почти что казино лотерея
2025-07-20 02:45:39,716 - INFO - Начало предобработки текста: 3-

Сообщение: Как же надоели эти сообщения про казино
Класс: 0
Вероятности: [0.9393701, 0.0606299]

Сообщение: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru
Класс: 0
Вероятности: [0.9999915, 8.5e-06]

Сообщение: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
Класс: 0
Вероятности: [0.9999915, 8.5e-06]

Сообщение: Поступление – это почти что казино! Лотерея!
Класс: 0
Вероятности: [0.9929106, 0.0070894]

Сообщение: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
Класс: 0
Вероятности: [0.989933, 0.010067]

Сообщение: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
Класс: 0
Вероятности: [0.9578892, 0.0421108]



### Naive Bayes + SMOTE

In [41]:
y_pred = model_nb_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.60      1.00      0.75      6689
           1       1.00      0.35      0.52      7019

    accuracy                           0.67     13708
   macro avg       0.80      0.68      0.64     13708
weighted avg       0.80      0.67      0.63     13708

Confusion Matrix:
 [[6687    2]
 [4530 2489]]





In [42]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-07-20 02:45:39,743 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-07-20 02:45:39,744 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-07-20 02:45:39,751 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-07-20 02:45:39,752 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-07-20 02:45:39,757 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-07-20 02:45:39,758 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-07-20 02:45:39,760 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-07-20 02:45:39,761 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-07-20 02:45:39,764 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.999113, 0.000887]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9880727, 0.0119273]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9969457, 0.0030543]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.8503812, 0.1496188]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9999796, 2.04e-05]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.9923

## Сохранение результатов

In [43]:
with open('../models/vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

with open('../models/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Векторизатор и масштабатор сохранены!")

Векторизатор и масштабатор сохранены!


In [44]:
with open('../models/lr_model.pkl', 'wb') as lr_model_file:
    pickle.dump(model_lr, lr_model_file)

print("Модель логической регрессии без семплирования сохранена!")

Модель логической регрессии без семплирования сохранена!


In [45]:
with open('../models/lr_sm_model.pkl', 'wb') as lr_sm_model_file:
    pickle.dump(model_lr_sm, lr_sm_model_file)

print("Модель логической регрессии со семплированием сохранена!")

Модель логической регрессии со семплированием сохранена!


In [46]:
with open('../models/rf_model.pkl', 'wb') as rf_model_file:
    pickle.dump(model_rf, rf_model_file)

print("Модель случайного леса без семплирования сохранена!")

Модель случайного леса без семплирования сохранена!


In [47]:
with open('../models/rf_sm_model.pkl', 'wb') as rf_sm_model_file:
    pickle.dump(model_rf_sm, rf_sm_model_file)

print("Модель случайного леса со семплированием сохранена!")

Модель случайного леса со семплированием сохранена!


In [48]:
with open('../models/nb_model.pkl', 'wb') as nb_model_file:
    pickle.dump(model_nb, nb_model_file)

print("Модель Naive Bayes без семплирования сохранена!")

Модель Naive Bayes без семплирования сохранена!


In [49]:
with open('../models/nb_sm_model.pkl', 'wb') as nb_sm_model_file:
    pickle.dump(model_nb_sm, nb_sm_model_file)

print("Модель Naive Bayes со семплированием сохранена!")

Модель Naive Bayes со семплированием сохранена!
